In [77]:
import pandas as pd
import nltk
import requests
import re
import json

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

In [2]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:
conf = sddk.configure()

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
occupations = gc.open_by_url("https://docs.google.com/spreadsheets/d/1nONTEwp42CVnq3iCiONrFbJedIcYtBV-l4Bil5mU7Eo/edit?usp=sharing")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [3]:
# login to our project folder, owned by my AU account 648597@au.dk, using my account at another institution
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [4]:
# read edh dataset
publicfolder = "b6b6afdb969d378b70929e86e58ad975"
EDH = sddk.read_file("EDH_text_cleaned_2021-01-21.json", "df", publicfolder)
EDH.head(5)

reading file located in a public folder


,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,D M Noniae P f Optatae et C Iulio Artemoni par...,Dis Manibus Noniae Publi filiae Optatae et Cai...,Dis Manibus Noniae Publi filiae Optatae et Cai...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",NaN,...,C Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,"Via Nomentana, S. Alessandro, Kirche",1937,NaN,NaN,NaN,NaN,NaN
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,ummio isenna Xv,Publio Mummio Publi filio Galeria Sisennae Rut...,Publio Mummio Publi filio Galeria Sisennae Rut...,NaN,before 1975,"Sevilla, Privatbesitz",NaN,NaN,NaN,NaN
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,AVSLLA M Porci Nigri ser dominae Veneri aram p...,AVS LLA Marci Porci Nigri serva dominae Veneri...,AVS LLA Marci Porci Nigri serva dominae Veneri...,NaN,before 1979,"Carcabuey, Grupo Escolar",names of pagan deities,NaN,NaN,NaN
4,Feraudi,epitaph,NaN,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,NaN,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",NaN,...,l Successus L l Irenaeus C l Tyches unt renti f,libertus Successus Luci libertus Irenaeus Cai ...,libertus Successus Luci libertus Irenaeus Cai ...,Via Cupa (ehem. Vigna Nardi),NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# read edcs dataset
publicfolder = "1f5f56d09903fe259c0906add8b3a55e"
EDCS = sddk.read_file("EDCS_text_cleaned_2021-01-21.json", "df", publicfolder)
EDCS.head(5)

reading file located in a public folder


,EDCS-ID,publication,province,place,notes_dating,notes_references,notes_comments,inscription,inscription_stripped14,clean_text_interpretive_word,Links,language,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment
0,EDCS-31400030,"CIL 03, 12297",Achaia,?,,,,Leius,Leius,Leius,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDCS-24700151,"CIL 01, 02650 (p 1097) = IG-05-01, 00741 = ILL...",Achaia,Afesou,,,,// D(ecimi) Leivei D(ecimi) Leivei salve,// D(ecimi) Leivei D(ecimi) Leivei salve,Decimi Leivei Decimi Leivei salve,http://db.edcs.eu/epigr/partner.php?s_language...,GR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EDCS-24900077,"CIL 01, 00746 (p 944) = D 00867 = ILLRP 00374 ...",Achaia,Agia Triada / Merbaka / Midea,,,,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Quinto Caecilio Cai filio Metello imperatori I...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,-68,-68,officium/professio; ordo senatorius; tituli ho...,37.6381128,22.8052991,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,NaN,NaN
3,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Agios Athanasios / Photike,,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piissimo Caesari domino nostro G...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,309,313,Augusti/Augustae; litterae erasae; ordo equest...,39.4512182,20.7667673,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,NaN,NaN
4,EDCS-55701593,"AE 2009, 01286a",Achaia,Agios Donatos / Photike,,,,Cn(aeus) Atei(us),Cn(aeus) Atei(us),Cnaeus Ateius,NaN,NaN,NaN,NaN,sigilla impressa; tituli fabricationis,39.4759759,20.5069076,NaN,opus figlinae,NaN


In [128]:
# read declined occupations
with open("../data/occups_declined_dict.json", "r") as fp:
    occups_declined_dict = json.load(fp)

In [129]:
# check that our occupations are properly arranged (from the longest...)
list(occups_declined_dict.keys())[:10]

['exactor auri argenti et aeris',
 'inclusor auri et gemmarum',
 'refector pectinarius',
 'instructor parietum',
 'tribor argentarius',
 'carrocarpentarius',
 'dactilidiogliphus',
 'manceps salinarum',
 'manceps thermarum',
 'aequator monetae']

In [130]:
# sample for testing
EDH_sample = EDH[:5000]
len(EDH_sample)

5000

In [131]:
occups_declined_dict["plastes"]

['plastas',
 'plastarum',
 'plaste',
 'plastae',
 'plastis',
 'plastes',
 'plasten',
 'plasta',
 'plastam']

In [132]:
def extract_occup(inscription_text):
    occups_found = []
    for occup in occups_declined_dict.keys():
        for occup_morph in occups_declined_dict[occup]:
            try:
                occup_morph_N = inscription_text.lower().count(occup_morph)
                if occup_morph_N > 0:
                    occups_found.extend([occup] * occup_morph_N)
                    inscription_text = inscription_text.replace(occup_morph, "")
                
            except: pass
    return occups_found

In [117]:
%%time
EDH_sample["occups"] = EDH_sample["clean_text_interpretive_word"].apply(extract_occup)
EDH_occups_list = [el for sublist in EDH_sample["occups"].tolist() for el in sublist]
len(EDH_occups_list) 
# v1: 3.03s, 946 found
# once we remove after first match, then only 636
# but once we are sensitive to repeated occurences, we get 742
# when looking for within a lower case inscription, the number increases to 902

CPU times: user 8.72 s, sys: 4 ms, total: 8.72 s
Wall time: 8.72 s


/home/kasev/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



902

In [118]:
EDH_occups_list[:10]

['intestinus',
 'plastes',
 'plastes',
 'plastes',
 'psaltes',
 'negotiator',
 'plastes',
 'ustor',
 'fullo',
 'dendrophorus']

In [119]:
%%time
EDH["occups"] = EDH["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 2min 5s, sys: 312 ms, total: 2min 5s
Wall time: 2min 5s


In [120]:
EDH_occups_list = [el for sublist in EDH["occups"].tolist() for el in sublist]
print(len(EDH_occups_list))
print(nltk.FreqDist(EDH_occups_list).most_common(30))

11601
[('plastes', 1894), ('curator', 1891), ('ustor', 1169), ('cerdo', 843), ('psaltes', 818), ('faber', 563), ('uctor', 306), ('medicus', 256), ('pollio', 238), ('promus', 192), ('scriba', 177), ('sagittarius', 172), ('mercator', 140), ('aerarius', 111), ('negotiator', 110), ('agricola', 110), ('marmorarienses', 106), ('emporus', 103), ('tector', 85), ('lanius', 72), ('centonarius', 70), ('tesserarius', 69), ('vexillarius', 68), ('cocus', 65), ('figlus', 65), ('gladiator', 64), ('argentarius', 60), ('aedifex', 49), ('mensor', 47), ('scriptor', 44)]


first results (for comparison):

```[('curator', 2833), ('ustor', 2092), ('plastes', 1076), ('psaltes', 866), ('cerdo', 730), ('uctor', 592), ('faber', 458), ('promus', 320), ('sagittarius', 236), ('erarius', 217), ('scriba', 194), ('negotiator', 191), ('medicus', 137), ('tector', 136), ('centonarius', 132), ('emporus', 113), ('gladiator', 112), ('aerarius', 104), ('marmorarienses', 102), ('conditor', 76), ('vexillarius', 74), ('lanius', 72), ('scaenicus', 71), ('tesserarius', 70), ('argentarius', 65), ('mensor', 62), ('scriptor', 61), ('figlus', 56), ('nauta', 53), ('cellio', 52)]```

In [121]:
%%time
EDCS["occups"] = EDCS["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 9min 51s, sys: 1.19 s, total: 9min 52s
Wall time: 9min 52s


In [122]:
EDCS_occups_list = [el for sublist in EDCS["occups"].tolist() for el in sublist]
print(len(EDCS_occups_list))
print(nltk.FreqDist(EDCS_occups_list).most_common(30))

40509
[('plastes', 5934), ('curator', 4368), ('figlus', 4251), ('cerdo', 2905), ('ustor', 2863), ('psaltes', 2205), ('faber', 2162), ('pollio', 907), ('medicus', 723), ('uctor', 671), ('scriba', 558), ('mercator', 546), ('aerarius', 517), ('promus', 485), ('cocus', 437), ('figulus', 419), ('sagittarius', 376), ('marmorarienses', 360), ('lanius', 334), ('emporus', 316), ('agricola', 310), ('fullo', 279), ('vexillarius', 273), ('gladiator', 248), ('restio', 246), ('aedifex', 245), ('negotiator', 244), ('vitor', 242), ('tector', 230), ('centonarius', 220)]


# Generating sample for closer inspection

In [123]:
EDH.columns

Index(['responsible_individual', 'type_of_inscription', 'letter_size',
       'not_after', 'literature', 'work_status', 'height', 'diplomatic_text',
       'people', 'depth', 'material', 'type_of_monument', 'province_label',
       'width', 'transcription', 'country', 'uri', 'findspot_ancient',
       'last_update', 'modern_region', 'findspot_modern', 'language', 'id',
       'edh_geography_uri', 'commentary', 'trismegistos_uri', 'not_before',
       'external_image_uris', 'fotos', 'coordinates', 'idno_tm',
       'placenames_refs', 'text_edition', 'origdate_text', 'layout_execution',
       'layout_execution_text', 'support_objecttype',
       'support_objecttype_text', 'support_material', 'support_material_text',
       'support_decoration', 'keywords_term', 'keywords_term_text',
       'type_of_inscription_clean', 'type_of_inscription_certainty',
       'height_cm', 'width_cm', 'depth_cm', 'material_clean',
       'type_of_monument_clean', 'type_of_monument_certainty',
       'provi

In [124]:
EDH["occups_N"] = EDH["occups"].apply(len)

In [125]:
len(EDH[EDH["occups_N"] > 0])

7871

In [126]:
# create sample and export it to gsheet
EDH_occups_sample = EDH[EDH["occups_N"] > 0][:200][["id", "clean_text_interpretive_word", "occups"]]
set_with_dataframe(occupations.add_worksheet("EDH_occups_sample", 1, 1), EDH_occups_sample)

# Occupations over time
